In [1]:
import os
import re
import sys
import warnings
import numpy as np
import pandas as pd

from MFDFA import MFDFA

sys.path.append('../modules')
import estimate_hurst_mfdfa as estimate_mfdfa

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

In [2]:
input_path_raw = "../input_files/raw_data"
input_path_processed = "../input_files/processed_data"
input_path_data_dictionary = "../input_files/data_dictionary"
log_path = "../logs"
output_path = "../output_files"
input_generation_date = "2023-06-08"

## Load time series

In [3]:
df_stock_indexes = pd.read_csv("{}/df_stock_index_{}.csv".format(input_path_processed, re.sub("-", "", input_generation_date)), low_memory = False)
df_currencies = pd.read_csv("{}/df_currency_{}.csv".format(input_path_processed, re.sub("-", "", input_generation_date)), low_memory = False)

## Estimate Hurst exponent

In [4]:
estimate_mfdfa.estimate_hurst_mfdfa(
    y_data = df_stock_indexes[df_stock_indexes["symbol"] == "^GSPC"]["log_return"].values,
    start = 0.5,
    end = 3,
    points = 100,
    q_orders = [2, 3, 4],
    orders = [1, 2],
    log_path = log_path,
    log_filename = "log_hurst_local",
    verbose = 1
)

,q_order,dfa_degree,hurst
0,2,1,0.511244
1,2,2,0.504556
2,3,1,0.492858
3,3,2,0.483850
4,4,1,0.473770
5,4,2,0.463490


In [5]:
estimate_mfdfa.estimate_hurst_mfdfa_df(
    df_data = df_stock_indexes[(df_stock_indexes["symbol"] == "^GSPC")],
    points = 100,
    n_step = 11900,
    log_path = log_path,
    log_filename = "log_hurst_df",
    verbose = 1,
    arg_list = [[2, 3], [1]]
)

,q_order,dfa_degree,hurst,symbol,step,time_series
0,2,1,0.000000,^GSPC,0,log-return
1,3,1,0.000000,^GSPC,0,log-return
2,2,1,0.000000,^GSPC,0,absolute log-return
3,3,1,0.000000,^GSPC,0,absolute log-return
4,2,1,0.000000,^GSPC,0,log-return volatility
5,3,1,0.000000,^GSPC,0,log-return volatility
6,2,1,0.511877,^GSPC,11900,log-return
7,3,1,0.483408,^GSPC,11900,log-return
8,2,1,0.934024,^GSPC,11900,absolute log-return
9,3,1,0.899238,^GSPC,11900,absolute log-return


In [6]:
df_stock_hurst = estimate_mfdfa.estimate_hurst_mfdfa_global(
    df_data = df_stock_indexes,
    minimal_steps = 60,
    points = 100,
    n_step = 5,
    q_orders = [-2, -1, 0, 1, 2],
    orders = [1],
    log_path = log_path,
    log_filename = "log_hurst_global_stock",
    verbose = 1,
    tqdm_bar = True
)

100%|█████████████████████| 5/5 [7:23:44<00:00, 5324.87s/it]


In [7]:
df_stock_hurst.to_csv("{}/df_hurst_stock_index_{}.csv".format(input_path_processed, re.sub("-", "", input_generation_date)) , index = False)

In [8]:
df_currency_hurst = estimate_mfdfa.estimate_hurst_mfdfa_global(
    df_data = df_currencies,
    minimal_steps = 60,
    points = 100,
    n_step = 15,
    q_orders = [-2, -1, 0, 1, 2],
    orders = [1],
    log_path = log_path,
    log_filename = "log_hurst_global_currency",
    verbose = 1,
    tqdm_bar = True
)

100%|████████████████████████| 5/5 [24:55<00:00, 299.11s/it]


## Save Hurst data

In [9]:
df_currency_hurst.to_csv("{}/df_hurst_currency_{}.csv".format(input_path_processed, re.sub("-", "", input_generation_date)) , index = False)